In [ ]:
!pip install pydrive2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1) Auth in Colab
from google.colab import auth
auth.authenticate_user()

# 2) Use the Google API client
from googleapiclient.discovery import build

drive_service = build('drive', 'v3')


In [ ]:
# https://drive.google.com/drive/folders/162jBs8_Kull7c1wPD5oJOlqOnvFDL_Ka?usp=sharing
SOURCE_FOLDER_ID = "162jBs8_Kull7c1wPD5oJOlqOnvFDL_Ka"
folder_metadata = {
    'name': 'SatImageClassification',  # folder name in your Drive
    'mimeType': 'application/vnd.google-apps.folder',
    'parents': ['root']  # or specify another folder ID
}
folder_check = drive_service.files().list(
                q=(
                    f"'root' in parents "
                    f"and trashed=false "
                    f"and name='SatImageClassification' "
                    f"and mimeType='application/vnd.google-apps.folder'"
                ),
                fields='files(id, name)'
            ).execute()
existing_folders = folder_check.get('files', [])
DEST_FOLDER_ID = ''
if existing_folders :
  print(" destination folder already exists")
  DEST_FOLDER_ID = existing_folders[0]['id']
else:
        new_folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
        DEST_FOLDER_ID = new_folder['id']
        print("Created destination folder:", DEST_FOLDER_ID)





In [ ]:
def copy_top_level_files(drive_service, source_folder_id, dest_folder_id='root'):
    # 1) List all items in the source folder
    results = drive_service.files().list(
        q=f"'{source_folder_id}' in parents and trashed=false",
        fields='files(id, name, mimeType)'
    ).execute()

    items = results.get('files', [])
    print(f"Found {len(items)} items in source folder.")

    for item in items:
        file_id = item['id']
        file_name = item['name']
        mime_type = item['mimeType']

        # If it's a folder, skip or handle separately
        if mime_type == 'application/vnd.google-apps.folder':
            print(f"Skipping subfolder '{file_name}' in this non-recursive example.")
            continue

        # Copy the file to the destination folder
        body = {
            'name': file_name,
            'parents': [dest_folder_id]
        }

        new_file = drive_service.files().copy(
            fileId=file_id,
            body=body,
            fields='id, name'
        ).execute()

        print(f"Copied file '{file_name}' -> ID: {new_file['id']}")

# Example usage:
copy_top_level_files(drive_service, SOURCE_FOLDER_ID, DEST_FOLDER_ID)


In [ ]:
def copy_folder_recursive(drive_service, source_folder_id, dest_folder_id='root'):
    """
    Recursively copy the contents of source_folder_id into dest_folder_id.
    - If a subfolder with the same name exists in dest_folder_id, reuse it.
    - If a file with the same name exists in dest_folder_id, skip copying.
    """
    # 1) List all items in the source folder
    response = drive_service.files().list(
        q=f"'{source_folder_id}' in parents and trashed=false",
        fields='files(id, name, mimeType)'
    ).execute()
    items = response.get('files', [])

    for item in items:
        src_id = item['id']
        name = item['name']
        mime_type = item['mimeType']

        if mime_type == 'application/vnd.google-apps.folder':
            # ----- Handle Subfolder -----
            # Check if a folder with this name already exists in the destination
            folder_check = drive_service.files().list(
                q=(
                    f"'{dest_folder_id}' in parents "
                    f"and trashed=false "
                    f"and name='{name}' "
                    f"and mimeType='application/vnd.google-apps.folder'"
                ),
                fields='files(id, name)'
            ).execute()
            existing_folders = folder_check.get('files', [])

            if existing_folders:
                # Folder exists; reuse its ID
                existing_folder_id = existing_folders[0]['id']
                print(f"[Folder Exists] '{name}' -> Reusing ID: {existing_folder_id}")
                # Recursively copy its contents
                copy_folder_recursive(drive_service, src_id, existing_folder_id)
            else:
                # Create a new folder in the destination
                folder_body = {
                    'name': name,
                    'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [dest_folder_id]
                }
                new_folder = drive_service.files().create(
                    body=folder_body,
                    fields='id, name'
                ).execute()

                print(f"[Folder Created] '{name}' -> ID: {new_folder['id']}")
                # Recursively copy the contents
                copy_folder_recursive(drive_service, src_id, new_folder['id'])

        else:
            # ----- Handle File -----
            # Check if a file with this name already exists in the destination
            file_check = drive_service.files().list(
                q=(
                    f"'{dest_folder_id}' in parents "
                    f"and trashed=false "
                    f"and name='{name}'"
                ),
                fields='files(id, name)'
            ).execute()
            existing_files = file_check.get('files', [])

            if existing_files:
                # A file with this name already exists, so skip
                print(f"[Skip File] '{name}' already exists in destination.")
            else:
                # Copy the file to the destination
                copy_body = {
                    'name': name,
                    'parents': [dest_folder_id]
                }
                new_file = drive_service.files().copy(
                    fileId=src_id,
                    body=copy_body,
                    fields='id, name'
                ).execute()

                print(f"[File Copied] '{name}' -> ID: {new_file['id']}")


In [ ]:
copy_folder_recursive(drive_service, SOURCE_FOLDER_ID,DEST_FOLDER_ID)

In [ ]:
# https://drive.google.com/drive/folders/123jQOB9Ruwgb08voyyGpwZyw0CNP2Y0n?usp=sharing

SOURCE_nvid_FOLDER_ID = "123jQOB9Ruwgb08voyyGpwZyw0CNP2Y0n"
folder_metadata = {
    'name': 'Landsat8',  # folder name in your Drive
    'mimeType': 'application/vnd.google-apps.folder',
    'parents': ['root']  # or specify another folder ID
}
folder_check = drive_service.files().list(
                q=(
                    f"'root' in parents "
                    f"and trashed=false "
                    f"and name='Landsat8' "
                    f"and mimeType='application/vnd.google-apps.folder'"
                ),
                fields='files(id, name)'
            ).execute()
existing_folders = folder_check.get('files', [])
DEST_nvid_FOLDER_ID = ''
if existing_folders :
  print(" destination folder already exists")
  DEST_nvid_FOLDER_ID = existing_folders[0]['id']
else:
        new_folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
        DEST_nvid_FOLDER_ID = new_folder['id']
        print("Created destination folder:", DEST_nvid_FOLDER_ID)


In [ ]:
copy_folder_recursive(drive_service, SOURCE_nvid_FOLDER_ID,DEST_nvid_FOLDER_ID)